In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
directory = '../limpieza'

df = pd.read_csv(os.path.join(directory,"SIAP.csv"))

df.head()

,Año,Mes,Ciclo,Modalidad,Cultivo,Entidad,Distrito,Municipio,Superficie(ha)_Sembrada,Superficie(ha)_Cosechada,Superficie(ha)_Siniestrada,Producción,Rendimiento(udm/ha)
0,2023,Enero,Otoño-Invierno,Riego,Ajo,Aguascalientes,Aguascalientes,Asientos,25.0,0.0,0.0,0.0,0.0
1,2023,Enero,Otoño-Invierno,Riego,Ajo,Aguascalientes,Aguascalientes,Cosío,42.0,0.0,0.0,0.0,0.0
2,2023,Enero,Otoño-Invierno,Riego,Ajo,Aguascalientes,Aguascalientes,Pabellón de Arteaga,35.0,0.0,0.0,0.0,0.0
3,2023,Enero,Otoño-Invierno,Riego,Ajo,Aguascalientes,Aguascalientes,Rincón de Romos,50.0,0.0,0.0,0.0,0.0
4,2023,Enero,Otoño-Invierno,Riego,Ajo,Aguascalientes,Aguascalientes,Tepezalá,25.0,0.0,0.0,0.0,0.0
